[2025-06-07 Fiddler](https://thefiddler.substack.com/p/can-you-crack-the-roman-code)
====================

Fiddler
-------
This kind of problem is easier to solve with code than with thinking.

In [1]:
from functools import cache

@cache
def combinations(code, digits):
    count = 0
    for digit in digits:
        if code == digit:
            count += 1
        elif code[0:len(digit)] == digit:
            count += combinations(code[len(digit):], digits)
    return count

In [2]:
combinations("IIIIIIIIII",frozenset({"I","II","III"}))

274

Extra credit
------------

In [3]:
combinations("IIIVIIIVIIIVIII",frozenset({"I","II","III","IV","V","VI","VII","VIII"}))

4000